In [ ]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jraf (jraf-universidade-federal-de-pernambuco). Use `wandb login --relogin` to force relogin


True

In [2]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

# Initialize W&B
wandb.init(
    project="autoencoder_tcc",
    config={
        "patience": 5,
        "epochs": 250,
        "batch_size": 16,
        "encoding_dim": 128,
        "imgsz": 640
    }
)

2024-12-14 20:54:50.011438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734220491.125838   34391 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734220491.521931   34391 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 20:54:53.743563: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import tensorflow as tf

print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from build import build_autoencoder

autoencoder, encoder = build_autoencoder(
    input_shape=(wandb.config.imgsz, wandb.config.imgsz, 3),
    encoding_dim=wandb.config.encoding_dim,
    batch_size=wandb.config.batch_size
)

I0000 00:00:1734220528.064390   34391 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5529 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
# Print model summary
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (16, 640, 640, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (16, 640, 640, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (16, 320, 320, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (16, 320, 320, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (16, 160, 160, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (16, 160, 160, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (16, 80, 80, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (16, 819200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (16, 128)              │   104,857,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (16, 819200)           │   105,676,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (16, 80, 80, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (16, 160, 160, 128)    │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (16, 320, 320, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (16, 640, 640, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (16, 640, 640, 3)      │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,868,483 (804.40 MB)

 Trainable params: 210,868,483 (804.40 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import os

from tensorflow.keras.utils import image_dataset_from_directory

cwd = os.getcwd()
data_path = os.path.join(cwd, '..', '..', 'ships_v10i')
img_size = (wandb.config.imgsz, wandb.config.imgsz)

# Create directories for train data
images_dir = os.path.join(data_path, "cropped")

# Load dataset using image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    images_dir,
    labels=None,  # No labels as this is for autoencoder
    image_size=img_size,
    batch_size=wandb.config.batch_size,
    shuffle=True
)

to_autoencoder = lambda n: (n, n)


# Repeat the dataset for uninterrupted training
train_dataset = train_dataset.map(lambda x: to_autoencoder(x / 255.0)).repeat()
# .prefetch(buffer_size=tf.data.AUTOTUNE)

Found 3406 files.


In [7]:
autoencoder.compile(optimizer="adam", loss="mse")

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

# Add EarlyStopping callback for patience
early_stopping = EarlyStopping(
    # Monitors training loss (you can switch to 'val_loss' if validation data is used)
    monitor="loss",
    patience=wandb.config.patience,
    restore_best_weights=True,  # Restores the best weights after stopping
    verbose=1
)

In [9]:
total_images = len(os.listdir(images_dir))
steps_per_epoch = total_images // int(wandb.config.batch_size)

In [42]:
autoencoder.fit(
    train_dataset,
    epochs=wandb.config.epochs,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
    callbacks=[
        WandbMetricsLogger(),
        WandbModelCheckpoint("autoencoder.keras"),
        early_stopping,
    ],
)

Epoch 1/250


I0000 00:00:1734220542.734953   34788 service.cc:148] XLA service 0x39ebbac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734220542.739870   34788 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2024-12-14 20:55:44.464734: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734220545.410962   34788 cuda_dnn.cc:529] Loaded cuDNN version 90300


In [ ]:
# Finish the W&B run
wandb.finish()

In [ ]:
from cluster import cluster_latent_space, plot_silhouette_scores

In [ ]:
latent_features = encoder.predict(train_dataset)

In [ ]:
results = cluster_latent_space(
    encoder, latent_features, min_clusters=4, max_clusters=32)

best_k = results["best_k"]
silhouette_scores = results["silhouette_scores"]
final_labels = results["final_labels"]

In [ ]:
plot_silhouette_scores(silhouette_scores, min_clusters=4)